Se requiere obtener información de las "peliculas", el "país" donde se realizo la grabación y cual fue la "Productora" encargada de realizarlo. La inforamción debe ser a partir del año 2010 en adelante de la fecha de lanzamiento, ordenado de manera ascendente por el "título" de la película.

- De la película se requiere mostrar el "título", "el presupuesto", "los ingresos obtenidos", "tiempo de duración" y la "fecha de lanzamiento".
- De Pais se requiere mostrar el "nombre del país".
- De la "Productora" se requiere el "nombre de la productora"

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Obtener las Peliculas

In [0]:
movie_df = spark.read.format("delta").load(f"{silver_foler_path}/movies").where("year_release_date >= 2010") \
                                .filter(f"file_date = '{v_file_date}'")

#### Obtener los DF de Paises

In [0]:
country_df = spark.read.format("delta").load(f"{silver_foler_path}/country")
production_country_df = spark.read.format("delta").load(f"{silver_foler_path}/production_country") \
                                .filter(f"file_date = '{v_file_date}'")

#### Obtener los DF de Compañia

In [0]:
company_df = spark.read.format("delta").load(f"{silver_foler_path}/production_company") \
                                .filter(f"file_date = '{v_file_date}'")

production_company_df = spark.read.format("delta").load(f"{silver_foler_path}/movie_company") \
                                .filter(f"file_date = '{v_file_date}'")

#### Cruce de tablas

In [0]:
country_production_final_df = production_country_df.join(country_df, production_country_df.country_id == country_df.country_id, "inner").select(country_df.country_name, production_country_df.movie_id, country_df.country_id)

In [0]:
company_production_final_df = production_company_df.join(company_df, production_company_df.company_id == company_df.company_id, "inner").select(company_df.company_name, production_company_df.movie_id, production_company_df.company_id)

#### TABLA FINAL

In [0]:
from pyspark.sql.functions import lit

In [0]:
final_df = movie_df.join(country_production_final_df, movie_df.movie_id == country_production_final_df.movie_id, "inner") \
                                                    .join(company_production_final_df,movie_df.movie_id == company_production_final_df.movie_id, "inner") \
                                                    .select(movie_df.movie_id,"country_id","company_id",movie_df.title, movie_df.budget, movie_df.revenue, movie_df.duration_time, movie_df.release_date, country_production_final_df.country_name, company_production_final_df.company_name) \
                                                    .withColumn("created_date",lit(v_file_date)).orderBy(movie_df.title,ascend=True)

In [0]:
display(final_df)

In [0]:
#overwrite_partition(final_df, "movie_gold","results_country_prod_company", "created_date")

In [0]:
merge_condition = 'tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id AND tgt.company_id = src.company_id AND tgt.created_date = src.created_date'
merge_delta_lake(final_df, "movie_gold", "results_country_prod_company", gold_folder_path, merge_condition, "created_date")

In [0]:
%sql
SELECT * FROM movie_gold.results_country_prod_company